In [1]:
import fastf1
import os
from fastf1.core import Laps
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np

fastf1.Cache.enable_cache('f1_cache')  # Enable FastF1 cache

In [2]:
def collect_data(years, gp_names, save_path="f1_dataset.csv"):
    # Load existing data if present
    if os.path.exists(save_path):
        existing_df = pd.read_csv(save_path)
        existing_keys = set(zip(existing_df['Year'], existing_df['GP']))
        print(f"🔁 Loaded existing dataset with {len(existing_df)} samples.")
    else:
        existing_df = pd.DataFrame()
        existing_keys = set()

    all_data = []

    for year in years:
        for gp in gp_names:
            if (year, gp) in existing_keys:
                print(f"⏭️ Skipped {year} {gp} (already fetched)")
                continue

            try:
                print(f"\n📦 Loading {year} {gp}...")

                q_session = fastf1.get_session(year, gp, 'Q')
                r_session = fastf1.get_session(year, gp, 'R')
                q_session.load(telemetry=False)
                r_session.load(telemetry=False)

                quali_laps = q_session.laps.pick_quicklaps()
                if quali_laps.empty:
                    raise ValueError("No lap data available")

                quali_laps = quali_laps.reset_index(drop=True)

                best_laps = quali_laps.sort_values("LapTime").groupby("Driver", as_index=False).first()
                best_laps = best_laps[[
                    'Driver', 'Team', 'LapTime', 'Compound', 'TyreLife', 'FreshTyre',
                    'Stint', 'TrackStatus', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Deleted'
                ]]

                for sec in ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']:
                    best_laps[f'{sec}_sec'] = best_laps[sec].dt.total_seconds()

                best_laps = best_laps.dropna(subset=['LapTime_sec'])

                results = r_session.results.reset_index(drop=True)
                if 'Abbreviation' in results.columns:
                    results = results.rename(columns={'Abbreviation': 'Driver'})
                else:
                    raise ValueError("Missing 'Abbreviation' column in race results")

                results = results[['Driver', 'Position', 'GridPosition']]
                results['Position'] = pd.to_numeric(results['Position'], errors='coerce')
                results['GridPosition'] = pd.to_numeric(results['GridPosition'], errors='coerce')

                merged = pd.merge(best_laps, results, on='Driver', how='left')
                merged['GP'] = gp
                merged['Year'] = year

                all_data.append(merged)

            except Exception as e:
                print(f"❌ Skipped {year} {gp} due to error: {e}")
                continue

    if all_data:
        new_df = pd.concat(all_data, ignore_index=True)
        full_df = pd.concat([existing_df, new_df], ignore_index=True)
        full_df.to_csv(save_path, index=False)
        print(f"✅ Updated dataset saved to {save_path}")
    else:
        full_df = existing_df
        print("⚠️ No new data collected.")

    return full_df

In [3]:
# Example usage
years = [2022, 2023, 2024]

races = [
    'Bahrain Grand Prix',
    'Saudi Arabian Grand Prix',
    'Australian Grand Prix',
    'Azerbaijan Grand Prix',
    'Miami Grand Prix',
    'Spanish Grand Prix',
    'Monaco Grand Prix',
    'Canadian Grand Prix',
    'British Grand Prix',
    'Hungarian Grand Prix',
    'Belgian Grand Prix',
    'Dutch Grand Prix',
    'Italian Grand Prix',
    'Singapore Grand Prix',
    'Japanese Grand Prix',
    'United States Grand Prix',
    'Mexico City Grand Prix',
    'São Paulo Grand Prix',
    'Las Vegas Grand Prix',
    'Abu Dhabi Grand Prix'
]

df = collect_data(years, races)
print("Total Samples:", len(df))
print(df.head())

🔁 Loaded existing dataset with 665 samples.
⏭️ Skipped 2022 Bahrain Grand Prix (already fetched)
⏭️ Skipped 2022 Saudi Arabian Grand Prix (already fetched)
⏭️ Skipped 2022 Australian Grand Prix (already fetched)
⏭️ Skipped 2022 Azerbaijan Grand Prix (already fetched)
⏭️ Skipped 2022 Miami Grand Prix (already fetched)
⏭️ Skipped 2022 Spanish Grand Prix (already fetched)
⏭️ Skipped 2022 Monaco Grand Prix (already fetched)
⏭️ Skipped 2022 Canadian Grand Prix (already fetched)
⏭️ Skipped 2022 British Grand Prix (already fetched)
⏭️ Skipped 2022 Hungarian Grand Prix (already fetched)
⏭️ Skipped 2022 Belgian Grand Prix (already fetched)
⏭️ Skipped 2022 Dutch Grand Prix (already fetched)
⏭️ Skipped 2022 Italian Grand Prix (already fetched)
⏭️ Skipped 2022 Singapore Grand Prix (already fetched)
⏭️ Skipped 2022 Japanese Grand Prix (already fetched)
⏭️ Skipped 2022 United States Grand Prix (already fetched)
⏭️ Skipped 2022 Mexico City Grand Prix (already fetched)
⏭️ Skipped 2022 São Paulo Grand 

core           INFO 	Loading data for Mexico City Grand Prix - Qualifying [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            I


📦 Loading 2023 São Paulo Grand Prix...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to


📦 Loading 2023 Las Vegas Grand Prix...


...


📦 Loading 2024 Las Vegas Grand Prix...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to


📦 Loading 2024 Abu Dhabi Grand Prix...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

✅ Updated dataset saved to f1_dataset.csv
Total Samples: 1130
  Driver        Team                 LapTime Compound  TyreLife  FreshTyre  \
0    ALB    Williams  0 days 00:01:32.664000     SOFT       2.0       True   
1    ALO      Alpine  0 days 00:01:31.621000     SOFT       2.0       True   
2    BOT  Alfa Romeo  0 days 00:01:31.560000     SOFT       2.0       True   
3    GAS  AlphaTauri  0 days 00:01:31.635000     SOFT       2.0       True   
4    HAM    Mercedes  0 days 00:01:31.048000     SOFT       2.0       True   

   Stint TrackStatus             Sector1Time             Sector2Time  \
0    5.0           1  0 days 00:00:29.564000  0 days 00:00:39.897000   
1    4.0           1  0 days 00:00:29.306000  0 days 00:00:39.305000   
2    5.0           1  0 days 00:00:29.203000  0 days 00:00:39.296000   
3    5.0           1  0 days 00:00:29.253000  0 days 00:00:39.335000   
4    3.0           1  0 days 00:00:29.100000  0 days 00:00:39.038000   

              Sector3Time  Deleted  

In [4]:
# Feature selection
features = [
    'Driver', 'Team', 'Compound', 'TyreLife', 'FreshTyre', 'Stint', 'TrackStatus',
    'Deleted', 'Sector1Time_sec', 'Sector2Time_sec', 'Sector3Time_sec', 'LapTime_sec', 'GridPosition'
]
target = 'Position'

# Categorical and numerical column lists
categorical = ['Driver', 'Team', 'Compound', 'FreshTyre', 'TrackStatus', 'Deleted']
numerical = ['TyreLife', 'Stint', 'Sector1Time_sec', 'Sector2Time_sec', 'Sector3Time_sec', 'LapTime_sec', 'GridPosition']

# Drop rows with missing target
df = df.dropna(subset=['Position'])

# Define X and y
X = df[features]
y = df[target]

In [6]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Ensure all categorical columns are strings
for col in categorical:
    X[col] = X[col].astype(str)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Full pipeline with placeholder regressor

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical),
    ('cat', cat_pipeline, categorical)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Hyperparameter grid
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}

# Grid search
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_
print("✅ Best Params:", grid_search.best_params_)

# Predict and evaluate
y_pred = best_model.predict(X_test)

print("\n📊 Tuned Model Performance:")
print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")


/tmp/ipykernel_11925/4089456476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(str)
/tmp/ipykernel_11925/4089456476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(str)
/tmp/ipykernel_11925/4089456476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

Fitting 5 folds for each of 24 candidates, totalling 120 fits


✅ Best Params: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 100}

📊 Tuned Model Performance:
MSE: 19.96
MAE: 3.45
R² Score: 0.44


In [7]:
rounded_preds = np.round(y_pred)

results_df = X_test.copy()
results_df['True_Position'] = y_test
results_df['Predicted_Position'] = rounded_preds.round(2)
print("\nSample Predictions:")
print(results_df[['Driver', 'Team', 'LapTime_sec', 'True_Position', 'Predicted_Position']].head(5))


Sample Predictions:
    Driver          Team  LapTime_sec  True_Position  Predicted_Position
904    ALO  Aston Martin       86.730            8.0                11.0
361    RUS      Mercedes       84.511            5.0                 6.0
56     TSU    AlphaTauri      103.056           13.0                10.0
714    OCO        Alpine       94.834            4.0                14.0
741    STR  Aston Martin       84.405           10.0                14.0


In [8]:
import joblib

# Save the best model
joblib.dump(best_model, 'f1_position_predictor.pkl')
print("✅ Model saved as 'f1_position_predictor.pkl'")


✅ Model saved as 'f1_position_predictor.pkl'
